In [1]:
import torch
from torchvision import models
from torchvision import transforms
import time

In [2]:
model = models.quantization.googlenet(
    weights=True, quantize=True)


/home/venom/miniconda3/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=GoogLeNet_QuantizedWeights.IMAGENET1K_FBGEMM_V1`. You can also use `weights=GoogLeNet_QuantizedWeights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/home/venom/miniconda3/lib/python3.8/site-packages/torch/ao/quantization/utils.py:280: UserWarning: must run observer before calling calculate_qparams. Returning default values.
  warnings.warn(


In [3]:
model.eval()

QuantizableGoogLeNet(
  (conv1): QuantizableBasicConv2d(
    (conv): QuantizedConvReLU2d(3, 64, kernel_size=(7, 7), stride=(2, 2), scale=0.08655554801225662, zero_point=0, padding=(3, 3))
    (bn): Identity()
    (relu): Identity()
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): QuantizableBasicConv2d(
    (conv): QuantizedConvReLU2d(64, 64, kernel_size=(1, 1), stride=(1, 1), scale=0.05618245154619217, zero_point=0)
    (bn): Identity()
    (relu): Identity()
  )
  (conv3): QuantizableBasicConv2d(
    (conv): QuantizedConvReLU2d(64, 192, kernel_size=(3, 3), stride=(1, 1), scale=0.05060958117246628, zero_point=0, padding=(1, 1))
    (bn): Identity()
    (relu): Identity()
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): QuantizableInception(
    (branch1): QuantizableBasicConv2d(
      (conv): QuantizedConvReLU2d(192, 64, kernel_size=(1, 1), stride=(1, 1), scale=0.053043022

In [4]:
# Load the image, resize it and convert it to a tensor and display it
from PIL import Image

img = Image.open('cat.jpg')
img = transforms.Resize((224, 224))(img)
img = transforms.ToTensor()(img)
# The image should be normalized according to the pytorch docs
img = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[
                           0.229, 0.224, 0.225])(img)


In [5]:
# Run the model on the image
start = time.time()
out = model(img.unsqueeze(0))
end = time.time()
print('Time taken: ', end - start)

Time taken:  0.023660659790039062


In [6]:
# Running inference 100 times
start = time.time()
for _ in range(100):
    output = model(img.unsqueeze(0))
end = time.time()
print(f'Time: {end - start} seconds')


Time: 0.7461836338043213 seconds


In previous notebook, single image inference took 0.0512 sec.
Now its 0.0363 sec

Around 1.4x faster

For the 100 times inference, it took 2.582 sec earlier
After quantization, it took 0.7035 sec

Around 3.7x faster

## Fusing layers

https://spell.ml/blog/pytorch-quantization-X8e7wBAAACIAHPhT

This is a really good article that goes over pytorch model quantization, it also goes over fusing layers, which improves the performance of the model, when used in python.